# WEEK 2

## Sample Questions

#### Q1. set up simple graph relating x,y and z

In [ ]:
import numpy as np
import pandas as pd 
import torch

In [12]:
import torch

x=torch.tensor(3.5,requires_grad=True)

y=x*x

z=2*y+3

print("x: ",x)
print("\ny=x*x: ",y)
print("\nz=2*y+3: ",z)

# work out gradients
z.backward()

print("\nWorking out gradients dz/dx")

# what is gradient at x=3.5 ?
print("Gradient at x=3.5: ",x.grad)

x:  tensor(3.5000, requires_grad=True)

y=x*x:  tensor(12.2500, grad_fn=<MulBackward0>)

z=2*y+3:  tensor(27.5000, grad_fn=<AddBackward0>)

Working out gradients dz/dx
Gradient at x=3.5:  tensor(14.)


#### Q2 consider the function f(x)=(x-2)^2. compute d(f(x))/dx and then compute f'(1). write code  and check analytical gradient

In [11]:
def f(x):
    return (x-2)**2
def fp(x):
    return 2*(x-2)

x=torch.tensor([1.0],requires_grad=True)

# pytorch
y=f(x)
y.backward()

print("Analytical derivative f'(x): ", fp(x))
print("Pytorch's derivative f'(x): ",x.grad)

Analytical derivative f'(x):  tensor([-2.], grad_fn=<MulBackward0>)
Pytorch's derivative f'(x):  tensor([-2.])


#### Q3. define a function y=x^2+5. The function y will not only carry the result of evaluating x, but also gradient function dely/delx called grad_fn in the new tensor y. compare the result with analytical gradient

In [16]:
import torch

x=torch.tensor([2.0])
x.requires_grad_(True)

y=x**2+5

print("\ny: ",y)

y.backward() #dy/dx
print("\nPyTorch Gradient: ",x.grad)

# Analytical 
dy_dx=2*x
print("\nAnalytical Gradient: ",dy_dx)




y:  tensor([9.], grad_fn=<AddBackward0>)

PyTorch Gradient:  tensor([4.])

Analytical Gradient:  tensor([4.], grad_fn=<MulBackward0>)


#### Q4 Write a fucntion to compute the gradient of the sigmoid function 

##### sigma(x)=s(c(b(a(x)))) where a(x)=-x
##### b(a)=e^a
##### c(b)=1+b
##### s(c)=1/c


In [20]:
import torch

def grad_sigmoid_manual(x):
    a=-x
    b=np.exp(a)
    c=1+b
    s=1.0/c

    ds_dc=(-1.0/(c**2))
    ds_db=ds_dc*1 # ds_dc* dc_db
    ds_da=ds_db*np.exp(a) # ds_da=ds_dc*dc_db*db_da
    ds_dx=ds_da*(-1) # ds_dx=ds_dc*dc_db*db_da*da_dx

    return ds_dx

def sigmoid(x):
    y=1.0 /(1.0+ torch.exp(-x))
    return y

input_x=2.0

x=torch.tensor(input_x).requires_grad_(True)
y=sigmoid(x)
y.backward()

# comparing results of manual and automatic gradient functions
print("autograd: ",x.grad.item())
print("\nmanual: ", grad_sigmoid_manual(input_x))

autograd:  0.10499356687068939

manual:  0.1049935854035065


## Excercise

#### Q1. draw computation graph and work out the gradient dz/da by following the path back from z to a and compare the result with the analytical gradient

In [26]:
import torch

input_a=2.0
input_b=3.0

a=torch.tensor(input_a, requires_grad=True)
b=torch.tensor(input_b, requires_grad=True)

x=2*a+3*b
y=5*a*a + 3*b*b*b
z=2*x+3*y

def manual(a,b):
    x=2*a+3*b
    y=5*a*a + 3*b**3
    z=2*x+3*y
                        #dz_dx=2 #dx_da=2
    dz_da= 2*2 + 3*5*2*a  #dz_da=dz_dx*dx_da+ dz_dy*dy_da

    dz_db=2*3*1 + 3*3*3*b**2

    return dz_da,dz_db


print("x: ",x)
print("y: ",y)
print("z: ",z)

print(f"\nManual Analytical Derivative at a={input_a} and b={input_b} :{manual(input_a,input_b)}")

z.backward()
print(f"\nPytorch's derivatives at a={a} and b={b}: ")
print("dz_da: ",a.grad)
print("dz_db: ",b.grad)


x:  tensor(13., grad_fn=<AddBackward0>)
y:  tensor(101., grad_fn=<AddBackward0>)
z:  tensor(329., grad_fn=<AddBackward0>)

Manual Analytical Derivative at a=2.0 and b=3.0 :(64.0, 249.0)

Pytorch's derivatives at a=2.0 and b=3.0: 
dz_da:  tensor(64.)
dz_db:  tensor(249.)


#### Q2. same question


In [39]:
def manual(b,x,w):
    u=w*x
    v=u+b
    a=max(0.0, v)

    if a==0:
        return 0
    else:
        da_dv=1
        da_du=1   #da_dv*dv_du
        da_db=1   #da_dv*dv_db
        da_dw=x   #da_dv*dv_du*du_dw
        da_dx=w   #da_dv*dv_du*du_dx

    return da_dw,da_db,da_dx

inp_x = 3.0
inp_w = 4.0
inp_b = 5.0

x = torch.tensor(inp_x,requires_grad=True)
w = torch.tensor(inp_w,requires_grad = True)
b = torch.tensor(inp_b,requires_grad = True)

u = w*x;u.retain_grad()
v = u + b;v.retain_grad()
relu = torch.nn.ReLU();
a = relu(v);a.retain_grad()

a.backward()
print("PyTorch Derivatives: ")
print("da_dw: ",w.grad)
print("da_db: ",b.grad)
print("da_dx: ",x.grad)

grad_w,grad_b,grad_x= manual(inp_b,inp_x,inp_w)
print("\nManual da_dw: ",grad_w)
print("Manual da_db: ",grad_b)
print("Manual da_dx: ",grad_x)

PyTorch Derivatives: 
da_dw:  tensor(3.)
da_db:  tensor(1.)
da_dx:  tensor(4.)

Manual da_dw:  3.0
Manual da_db:  1
Manual da_dx:  4.0


#### Q3. Same Problem sigmoid Function

#### Q4. analytical gradient and analytical gradient of function =e^(-x^2-2*x-sin(x)) wrt x

In [ ]:
def manual(x):
    

#### Q5. same question. function is 8*x**4+ 3*x**3 + 7*x**2 +6*x +3

In [44]:
def manual(x):
    y=8*x**4+ 3*x**3 + 7*x**2 +6*x +3
    dy_dx=32*(x**3) + 9*(x**2) + 14*x + 6
    return dy_dx
    
inp_x=3.0
x = torch.tensor(inp_x,requires_grad=True)
a=8*x**4+ 3*x**3 + 7*x**2 +6*x +3

a.backward()

print("PyTorch Derivatives: ")
print("da_dx: ",x.grad)

print("Manual da_dx: ",manual(inp_x))

PyTorch Derivatives: 
da_dx:  tensor(993.)
Manual da_dx:  993.0


#### Q6. 